# Rodar algortimos de Machine Learning

In [1]:
# Libs
import pandas as pd

# Importa Classes de objetos
from analise_qualidade_vinhos.pipeline.model_builder import build_training_pipeline


#### Extrair dados para o uso do modelo

In [2]:
pipeline = build_training_pipeline(algorithm='random_forest', balance_method='smoteenn')


In [3]:
pipeline.fit(X_train, y_train)
preds = pipeline.predict(X_test)

NameError: name 'X_train' is not defined